<a href="https://colab.research.google.com/github/mridul-eecs/signal-processing-apachesparkml-apachesystemml/blob/master/KmeansSparkML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Preprocessing to work with spark

In [1]:

# spark dependencies:
# citation: http://medium.com/@rmache/big-data-with-spark-in-google-colab-7c046e24b3
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://apachemirror.wuchna.com/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz

!pip install -q findspark
!pip install pyspark

     |████████████████████████████████| 215.7MB 61kB/s 
     |████████████████████████████████| 204kB 49.6MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.4-py2.py3-none-any.whl size=216130387 sha256=509b28298180f529865da2c474e39e54e07fd3ea8833dc8947c7232be7542392
  Stored in directory: /root/.cache/pip/wheels/ab/09/4d/0d184230058e654eb1b04467dbc1292f00eaa186544604b471
Successfully built pyspark


In [2]:
!git clone https://github.com/mridul-eecs/signal-processing-apachesparkml-apachesystemml.git

Cloning into 'signal-processing-apachesparkml-apachesystemml'...
remote: Enumerating objects: 44, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 44 (delta 12), reused 16 (delta 2), pack-reused 0
Unpacking objects: 100% (44/44), done.


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, IntegerType
from pyspark.sql.functions import lit
import os
from tqdm import tqdm_notebook as tqdm

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"


import findspark
findspark.init()

APP_NAME= "Human Motion Premitives"
SPARK_URL= "local[*]"
RANDOM_SEED = 141109
TRAINING_DATA_RATIO = 0.7
RF_NUM_TREES = 8
RF_MAX_DEPTH = 4
RF_NUM_BINS = 32

In [0]:
spark= SparkSession.builder.appName(APP_NAME).master(SPARK_URL).getOrCreate()

### Main program

In [0]:
datapath= '/content/signal-processing-apachesparkml-apachesystemml/df.parquet'
df= spark.read.parquet(datapath)

In [0]:
from pyspark.ml.clustering import KMeans
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorAssembler, Normalizer

vectorizer= VectorAssembler(inputCols= ['x', 'y', 'z'], outputCol= 'features')
kmeans= KMeans().setK(13).setSeed(1)
pipe= Pipeline(stages= [vectorizer, kmeans])

model= pipe.fit(df)


In [15]:
model.transform(df)

DataFrame[x: int, y: int, z: int, class: string, features: vector, prediction: int]

In [0]:
wssse= model.stages[-1].computeCost(vectorizer.transform(df))

In [18]:
print(wssse)

15827497.774103818


In [24]:
# considering only 2 classes 
df= df.createOrReplaceTempView('df')
bindf= spark.sql("""select * from df where df.class in ('Use_telephone', 'Standup_chair')""")
bindf.show()

+---+---+---+-------------+
|  x|  y|  z|        class|
+---+---+---+-------------+
| 14| 46| 31|Standup_chair|
| 49| 24| 40|Use_telephone|
|  7| 30| 17|Standup_chair|
| 16| 41| 44|Standup_chair|
| 34| 43| 44|Use_telephone|
| 14| 40| 33|Standup_chair|
| 14| 40| 33|Standup_chair|
| 14| 40| 33|Standup_chair|
| 14| 40| 33|Standup_chair|
| 14| 40| 33|Standup_chair|
| 14| 40| 33|Standup_chair|
| 14| 40| 33|Standup_chair|
| 44| 31| 50|Use_telephone|
| 44| 31| 50|Use_telephone|
| 12| 30| 33|Standup_chair|
| 29| 41| 51|Standup_chair|
| 29| 41| 51|Standup_chair|
| 29| 41| 51|Standup_chair|
| 29| 41| 51|Standup_chair|
| 25| 36| 44|Use_telephone|
+---+---+---+-------------+
only showing top 20 rows



In [25]:
vectorizer= VectorAssembler(inputCols= ['x', 'y', 'z'], outputCol= 'features')
kmeans= KMeans().setK(2).setSeed(1)
pipe= Pipeline(stages= [vectorizer, kmeans])

model= pipe.fit(bindf)
pred= model.transform(bindf)
model.stages[-1].computeCost(vectorizer.transform(bindf))

4478194.572658991